***1. ОПИС***
1. Визначення сутностей та атрибутів:


**Підрозділи (Units)**

unit_id (INT, PRIMARY KEY): Унікальний ідентифікатор підрозділу.

unit_name (VARCHAR(255), NOT NULL): Назва підрозділу.

unit_location (VARCHAR(255)): Місце дислокації підрозділу.



**Операції (Operations)**

operation_id (INT, PRIMARY KEY): Унікальний ідентифікатор операції.

operation_name (VARCHAR(255), NOT NULL): Назва операції.

start_date (DATE): Дата початку операції.

end_date (DATE): Дата закінчення операції.

operation_status (VARCHAR(50)): Статус операції (наприклад, "Заплановано", "Виконується", "Завершено").


**Звіти (Reports)**

report_id (INT, PRIMARY KEY): Унікальний ідентифікатор звіту.

report_date (DATE, NOT NULL): Дата створення звіту.

description (TEXT): Опис звіту.

operation_id (INT, FOREIGN KEY): Ідентифікатор операції, до якої відноситься звіт.


**Підрозділи_Операції (Units_Operations)** -  Ця таблиця потрібна для зв'язку "багато-до-багатьох"

unit_id (INT, FOREIGN KEY): Ідентифікатор підрозділу.

operation_id (INT, FOREIGN KEY): Ідентифікатор операції.


2. Зв'язки між сутностями:


**Units - Operations: M:N (багато-до-багатьох)**

Один підрозділ може брати участь у багатьох операціях.

В одній операції може брати участь багато підрозділів.

Реалізується через проміжну таблицю Units_Operations.


**Operations - Reports: 1:N (один-до-багатьох)**

Одна операція може мати багато звітів.

Один звіт може стосуватися тільки однієї операції.


**Units - Reports:  1:N (один-до-багатьох)** - можливо, буде непрямий звязок через Operations

Один підрозділ може згадуватись у багатьох звітах.

Один звіт може стосуватися одного підрозділу (але звіт може стосуватися операцій, а в них задіяні  підрозділи, тому тут може бути не прямий звязок)


***2. SQL:***

In [ ]:
```mysql
-- Створення таблиці Units
CREATE TABLE Units (
    unit_id SERIAL PRIMARY KEY,
    unit_name VARCHAR(255) NOT NULL,
    unit_location VARCHAR(255)
);

-- Створення таблиці Operations
CREATE TABLE Operations (
    operation_id SERIAL PRIMARY KEY,
    operation_name VARCHAR(255) NOT NULL,
    start_date DATE,
    end_date DATE,
    operation_status VARCHAR(50)
);

-- Створення таблиці Reports
CREATE TABLE Reports (
    report_id SERIAL PRIMARY KEY,
    report_date DATE NOT NULL,
    description TEXT,
    operation_id INT REFERENCES Operations(operation_id)
);

-- Створення таблиці зв'язку Units_Operations (M:N)
CREATE TABLE Units_Operations (
    unit_id INT REFERENCES Units(unit_id),
    operation_id INT REFERENCES Operations(operation_id),
    PRIMARY KEY (unit_id, operation_id)
);
```

***3. PlantUML:***

In [ ]:
@startuml

entity "Units" as Units {
  * unit_id : INT <<PK>>
  --
  * unit_name : VARCHAR(255)
  unit_location : VARCHAR(255)
}

entity "Operations" as Operations {
  * operation_id : INT <<PK>>
  --
  * operation_name : VARCHAR(255)
  start_date : DATE
  end_date : DATE
  operation_status : VARCHAR(50)
}

entity "Reports" as Reports {
  * report_id : INT <<PK>>
  --
  * report_date : DATE
  description : TEXT
  operation_id : INT <<FK>>
}

entity "Units_Operations" as Units_Operations {
  * unit_id : INT <<FK>>
  * operation_id : INT <<FK>>
  --
  PRIMARY KEY (unit_id, operation_id)
}

' Зв'язки між сутностями
Units }o--o{ Units_Operations : "participates in (M:N)"
Operations }o--o{ Units_Operations : "involves (M:N)"
Operations }|--o{ Reports : "generates (1:N)"
Units ||.o{ Reports : "mentioned in (1:N)"

@enduml

***4. ERD PlantUML diagram:***

<img src="ERD_diagram.png">

In [ ]:
```mysql
-- Додавання підрозділів
INSERT INTO Units (unit_name, unit_location) VALUES
('1-ша окрема танкова бригада', 'Чернігів'),
('72-га окрема механізована бригада', 'Біла Церква'),
('93-тя окрема механізована бригада', 'Черкаське');

-- Додавання операцій
INSERT INTO Operations (operation_name, start_date, end_date, operation_status) VALUES
('Операція "Звільнення"', '2023-11-15', '2023-11-25', 'Завершено'),
('Операція "Оборона"', '2023-12-01', NULL, 'Виконується'),
('Операція "Контрнаступ"', '2024-01-10', NULL, 'Заплановано');

-- Додавання звітів
INSERT INTO Reports (report_date, description, operation_id) VALUES
('2023-11-20', 'Звіт про хід операції "Звільнення". Втрати: 2 танки, 5 осіб.', 1),
('2023-11-26', 'Підсумковий звіт операції "Звільнення".', 1),
('2023-12-05', 'Звіт про стан оборони.', 2);

-- Додавання зв'язків між підрозділами та операціями
INSERT INTO Units_Operations (unit_id, operation_id) VALUES
(1, 1),
(2, 1),
(2, 2),
(3, 3);
```